In [ ]:
import pandas as pd
import numpy as np
import datetime
import calendar
import random

!pip install jpholiday
import jpholiday

from IPython.display import display
#import matplotlib.pyplot as plt
#import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive


In [ ]:
#マスタデータ読み取り
input_path = "/content/drive/MyDrive/Colab Notebooks/data/日本鋳造/"

processes=pd.read_csv(input_path +"processes.csv",encoding="cp932")
person=pd.read_csv(input_path +"person.csv",encoding="cp932")
person=person.set_index(person.columns[0])
machineries=pd.read_csv(input_path +"machineries.csv",encoding="cp932")
machineries=machineries.set_index(machineries.columns[0])

order=pd.read_csv(input_path +"order_220d.csv",encoding="cp932")

In [ ]:
def input_order(order,processes):
#a=order.csv,b= processes.csv
  #列削除と名前変え
  order=order.dropna(how="all",axis=1).drop(labels=["顧客名","製品No","鋳込日"],axis=1).reset_index(drop=True)#test
  order=order[~order["納期"].str.contains("仮")].reset_index(drop=True)
  order=order.rename(columns={"品名": "merchandise","納期": "deadline"})
  order["days_left"]=np.nan#残り日保存用
  order["today"]=datetime.date.today()#計算用

  #日付typeに転換
  for i in range(len(order)):
    order.loc[i,"deadline"]=datetime.datetime.strptime(order.loc[i,"deadline"],"%Y/%m/%d")

  #count business day
  holiday_3years=jpholiday.year_holidays(datetime.date.today().year)+jpholiday.year_holidays(datetime.date.today().year+1)+jpholiday.year_holidays(datetime.date.today().year+2)
  list_holiday=[]
  for i in range(len(list(holiday_3years))):
    list_holiday.append(list(holiday_3years)[i][0])
  list_holiday=np.busdaycalendar(holidays=list_holiday)
  order["days_left"]=np.busday_count(order["today"].values.astype("datetime64[D]"),order["deadline"].values.astype("datetime64[D]"),busdaycal = list_holiday)
  order=order.drop(labels=["today"],axis=1)

  #processesと結合し、残り日に従って並び替え
  df_order = pd.merge(order,processes, on="merchandise")
  df_order["days_left"]=df_order["days_left"]-df_order["leadtime"]
  df_order.sort_values(by = "days_left", ascending = True, inplace = True)
  df_order = df_order.reset_index(drop=True)

  #見やすいため日付タイプをstrタイプに戻る
  for i in range(len(df_order)):
    df_order.loc[i,"deadline"]=df_order.loc[i,"deadline"].strftime("%Y/%m/%d")

  return df_order

In [ ]:
#test用order
# order=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/日本鋳造/order_test.csv",encoding="cp932")
# df_input=input_order(order.iloc[:3,:],processes)

#本番関数実行
df_input=input_order(order,processes)
#warningのflag作成
df_warning=df_input
df_warning["warning"]=df_warning["days_left"]<0
#df_warning=df_warning.loc[:,"warning"]
#df_warning=df_warning.loc[:,["merchandise","deadline","warning"]]

In [ ]:
df_input

,merchandise,deadline,days_left,leadtime,process1,process1_time,process1_time_cool,process1_a,process1_a_time,process1_a_time_cool,...,process17,process17_time,process17_time_cool,process17_a,process17_a_time,process17_a_time_cool,process17_b,process17_b_time,process17_b_time_cool,warning
0,ﾍﾞｰｽ,2022/11/25,-1.5,66.5,A20,2.0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,主眼筐体,2022/12/02,-1.0,71.0,A10,0.5,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,ﾍﾞｰｽ,2022/12/02,3.5,66.5,A20,2.0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,ﾍﾞｰｽ,2022/12/02,3.5,66.5,A20,2.0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,D-Scope鋳物,2022/11/17,5.0,55.0,A20,2.0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,30B下定盤,2022/12/17,62.0,19.0,A20,1.5,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
206,30B上定盤ｼﾞｬｹｯﾄ,2023/01/05,64.5,28.5,A20,3.5,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
207,EH3500TRA,2022/12/24,67.5,18.5,A10,0.5,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
208,90TKBｱｰﾑｻｽﾍﾟﾝｼｮﾝ,2023/04/10,111.0,46.0,A20,8.0,0,NaN,NaN,NaN,...,M24,6.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,False


In [ ]:
#output用columns作成

#日付と時間の生成
i=0
list_date=[]
#while len(list_date)<240:
#out put-> 30 business day
for i in range(31):
  #週末判断
  if (datetime.date.today()+datetime.timedelta(days=i)).weekday()!=5 and (datetime.date.today()+datetime.timedelta(days=i)).weekday()!=6:
    #祝日判断
    if jpholiday.is_holiday(datetime.date.today()+datetime.timedelta(days=i))== False:
      t=(datetime.date.today()+datetime.timedelta(days=i)).strftime("%Y/%m/%d")
      for j in range(1,17):
      #output 毎日8時間 2*8=16
        list_date.append(t+","+str(j))

#datetime.date.today()->開始日input可能の関数に切り替え

list_output=["merchandise","deadline"]
list_output=list_output+list_date

df_output=pd.DataFrame(columns=list_output,index=range(len(df_input)))
df_output["merchandise"]=df_input["merchandise"]
df_output["deadline"]=df_input["deadline"]
df_output=df_output.dropna(how="all")

In [ ]:
df_output

,merchandise,deadline,"2022/08/19,1","2022/08/19,2","2022/08/19,3","2022/08/19,4","2022/08/19,5","2022/08/19,6","2022/08/19,7","2022/08/19,8",...,"2022/09/16,7","2022/09/16,8","2022/09/16,9","2022/09/16,10","2022/09/16,11","2022/09/16,12","2022/09/16,13","2022/09/16,14","2022/09/16,15","2022/09/16,16"
0,ﾍﾞｰｽ,2022/11/25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,主眼筐体,2022/12/02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ﾍﾞｰｽ,2022/12/02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ﾍﾞｰｽ,2022/12/02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,D-Scope鋳物,2022/11/17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,30B下定盤,2022/12/17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,30B上定盤ｼﾞｬｹｯﾄ,2023/01/05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,EH3500TRA,2022/12/24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,90TKBｱｰﾑｻｽﾍﾟﾝｼｮﾝ,2023/04/10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def check_machineries_person(code,machineries,person):
    #x=process code,y=machineries.csv, z=person.csv
    machineries_process=machineries
    machineries_process=machineries_process[machineries_process["process"] == code]
    machineries_process=machineries_process[machineries_process["num"] > 0]#対応できる設備を抽出

    if machineries_process.empty==True:
      return False

    #"外注"を含まれているかどうかで内製と外注を区別する
    machineries_process_internal = machineries_process[~machineries_process.index.str.contains("外注")]
    machineries_process_external = machineries_process[machineries_process.index.str.contains("外注")]

    #check process
    if machineries_process_internal["num"].sum() != 0:#check内製可能な設備数は0ではない
        assignment_merchinery=machineries_process_internal.index[random.randint(0,len(machineries_process_internal.index)-1)]#設備ランダム振り分けと名称確認

        person_process=person.loc[:,[code,"num"]]#check対応者いるかどうか
        person_process=person_process[person_process[code]==1]#対応できる人を抽出
        if person_process["num"].sum() != 0:#check対応者人数
          person_process=person_process[person_process["num"]!=0]#num足りない作業員排除
          assignment_person=person_process.index[random.randint(0,len(person_process.index)-1)]#作業員ランダム振り分けと名称確認
          #両方check済み、okだったら、元のデータからnum変更し、returnする
          person.loc[assignment_person,"num"]=person.loc[assignment_person,"num"]-1
          machineries.loc[assignment_merchinery,"num"]=machineries.loc[assignment_merchinery,"num"]-1
          return [assignment_merchinery,assignment_person]
        else:#工程対応者いなかったら、check外注できるかどうか
          if machineries_process_external["num"].sum() != 0:
            assignment_merchinery=machineries_process_external.index[random.randint(0,len(machineries_process_external.index)-1)]
            machineries.loc[assignment_merchinery,"num"]=machineries.loc[assignment_merchinery,"num"]-1
            return [assignment_merchinery,""]
            #check済みで、OKだったらreturn外注工程名
          else:
            return False
    elif machineries_process_external["num"].sum() != 0:#check外注できるかどうか
      assignment_merchinery=machineries_process_external.index[random.randint(0,len(machineries_process_external.index)-1)]
      machineries.loc[assignment_merchinery,"num"]=machineries.loc[assignment_merchinery,"num"]-1
      return [assignment_merchinery,""]
    else:
      return False


In [ ]:
# #test用
# #マスタデータ読み取り
# input_path = "/content/drive/MyDrive/Colab Notebooks/data/日本鋳造/"

# processes=pd.read_csv(input_path +"processes.csv",encoding="cp932")
# person=pd.read_csv(input_path +"person.csv",encoding="cp932")
# person=person.set_index(person.columns[0])
# machineries=pd.read_csv(input_path +"machineries.csv",encoding="cp932")
# machineries=machineries.set_index(machineries.columns[0])

In [ ]:
#現在機械人空状況保存と計算用df:df_checkを作る
list_merchandise=df_input["merchandise"].tolist()
df_check=pd.DataFrame(columns=["merchandise","process_now","assignment_machinery","assignment_person","person_save","machinery_time","person_time","output_flag","process_flag"],index=range(len(list_merchandise)))
df_check["merchandise"]=df_input["merchandise"]
df_check["process_now"]=1
df_check["output_flag"]=False
df_check["process_flag"]=True

#loop横軸でoutput(df_output最初の2列はoutput保存対象外)
for date_index in range(len(df_output.columns)-2):
#for date_index in range(10):
  #loop縦軸で製品生産対応できるかどうかのcheck&output
  for i in range(len(list_merchandise)):#merchandise's num
  #製品対応工程抽出
    df_process=df_input[df_input["merchandise"]==list_merchandise[i]]
    df_a=df_process.filter(items=["process"+str(df_check.loc[i,"process_now"])])
    df_b=df_process.filter(like="process"+str(df_check.loc[i,"process_now"])+"_")
    df_process_spend=pd.concat([df_a,df_b],axis=1).reset_index()

    #check:"output_flag"=Falseの場合は次の工程が配分されられるかどうか
    if df_check.loc[i,"output_flag"]==False:
      #check:最大工程数は17のため、17以上超えるとbugが出る。process番号は17より大きい場合"process_flag"=False
      if df_check.loc[i,"process_flag"]==True:

        #工程は最大三つの選択肢があるため、優先順位で工程が入れるかどうかを判断
        #判断後は[工程の機械と人、それぞれかかる時間、output_flag=True]をdf_checkに保存
        assignment=check_machineries_person(df_process_spend.loc[0,"process"+str(df_check.loc[i,"process_now"])],machineries,person)
        if assignment!=False:
          person_time=df_process_spend.loc[0,"process"+str(df_check.loc[i,"process_now"])+"_time"]
          person_time=person_time/0.5
          df_check.loc[i,"person_time"]=person_time
          mechinery_time=df_process_spend.loc[0,"process"+str(df_check.loc[i,"process_now"])+"_time_cool"]
          mechinery_time=mechinery_time/0.5
          df_check.loc[i,"machinery_time"]=mechinery_time+person_time
          df_check.loc[i,"assignment_machinery"]=assignment[0]
          df_check.loc[i,"assignment_person"]=assignment[1]
          df_check.loc[i,"person_save"]=assignment[1]
          df_check.loc[i,"output_flag"]=True
        else:
          assignment=check_machineries_person(df_process_spend.loc[0,"process"+str(df_check.loc[i,"process_now"])+"_a"],machineries,person)
          if assignment!=False:
            person_time=df_process_spend.loc[0,"process"+str(df_check.loc[i,"process_now"])+"_a"+"_time"]
            person_time=person_time/0.5
            df_check.loc[i,"person_time"]=person_time
            mechinery_time=df_process_spend.loc[0,"process"+str(df_check.loc[i,"process_now"])+"_a"+"_time_cool"]
            mechinery_time=mechinery_time/0.5
            df_check.loc[i,"machinery_time"]=mechinery_time+person_time
            df_check.loc[i,"assignment_machinery"]=assignment[0]
            df_check.loc[i,"assignment_person"]=assignment[1]
            df_check.loc[i,"person_save"]=assignment[1]
            df_check.loc[i,"output_flag"]=True
          else:
            assignment=check_machineries_person(df_process_spend.loc[0,"process"+str(df_check.loc[i,"process_now"])+"_b"],machineries,person)
            if assignment!=False:
              person_time=df_process_spend.loc[0,"process"+str(df_check.loc[i,"process_now"])+"_b"+"_time"]
              person_time=person_time/0.5
              df_check.loc[i,"person_time"]=person_time
              mechinery_time=df_process_spend.loc[0,"process"+str(df_check.loc[i,"process_now"])+"_b"+"_time_cool"]
              mechinery_time=mechinery_time/0.5
              df_check.loc[i,"machinery_time"]=mechinery_time+person_time
              df_check.loc[i,"assignment_machinery"]=assignment[0]
              df_check.loc[i,"assignment_person"]=assignment[1]
              df_check.loc[i,"person_save"]=assignment[1]
              df_check.loc[i,"output_flag"]=True
            else:
              assignment=False
              df_check.loc[i,"output_flag"]=False

      #工程配分可能であれば、output_flagはTrueに変更することで、計算とoutputに入る
      if df_check.loc[i,"output_flag"]==True:
        if df_check.loc[i,"process_flag"]==True:
          if df_check.loc[i,"person_time"]>0:
            df_check.loc[i,"person_time"]=df_check.loc[i,"person_time"]-1
          elif df_check.loc[i,"person_time"]==0:
            df_check.loc[i,"assignment_person"]=""

          if df_check.loc[i,"machinery_time"]>0:
            df_check.loc[i,"machinery_time"]=df_check.loc[i,"machinery_time"]-1
            df_output.iloc[i,date_index+2]=df_check.loc[i,"assignment_machinery"]+","+df_check.loc[i,"assignment_person"]

    #check:"output_flag"=Trueの場合は計算とoutputを行う（二回目以後のloop）
    elif df_check.loc[i,"output_flag"]==True:
      if df_check.loc[i,"process_flag"]==True:

        if df_check.loc[i,"person_time"]>0:
          df_check.loc[i,"person_time"]=df_check.loc[i,"person_time"]-1
        elif df_check.loc[i,"person_time"]==0:
          df_check.loc[i,"assignment_person"]=""

        if df_check.loc[i,"machinery_time"]>0:
          df_check.loc[i,"machinery_time"]=df_check.loc[i,"machinery_time"]-1
          df_output.iloc[i,date_index+2]=df_check.loc[i,"assignment_machinery"]+","+df_check.loc[i,"assignment_person"]

  #計算終了後に、機械の数と人の数を戻す(+1)
  for i in range(len(list_merchandise)):
    if df_check.loc[i,"machinery_time"]==0:
      #if df_check.loc[i,"output_flag"]==True:
      machineries.loc[df_check.loc[i,"assignment_machinery"],"num"]=machineries.loc[df_check.loc[i,"assignment_machinery"],"num"]+1
      df_check.loc[i,"machinery_time"]=-1
      df_check.loc[i,"output_flag"]=False
      #process_nowを17以上に超えないように
      if df_check.loc[i,"process_now"]<=16:
        df_check.loc[i,"process_now"]=df_check.loc[i,"process_now"]+1
      elif df_check.loc[i,"process_now"]>16:
        df_check.loc[i,"process_flag"]=False
        df_check.loc[i,"output_flag"]=False
    if df_check.loc[i,"person_time"]==0:
      if df_check.loc[i,"person_save"] !="":
        person.loc[df_check.loc[i,"person_save"],"num"]=person.loc[df_check.loc[i,"person_save"],"num"]+1
        df_check.loc[i,"person_time"]=-1

In [ ]:
df_output.insert(0,"warning",df_warning["warning"])

In [ ]:
# #現在機械人空状況保存と計算用df
# df_check

In [ ]:
df_output

In [ ]:
#UI関連整形

df0=df_output
df1=df0
df_double=pd.concat([df0,df1])
df_double = df_double.sort_index()
df_double=df_double.reset_index(drop=True)

for i in range(len(df_double)):
  if i%2 == 0:
    for j in range(len(df_double.columns)-3):
      if type(df_double.iloc[i,3+j])!=float:
        df_double.iloc[i,3+j]=df_double.iloc[i,3+j].split(",")[0]
  elif i%2 == 1:
    for j in range(len(df_double.columns)-3):
      if type(df_double.iloc[i,3+j])!=float:
        df_double.iloc[i,3+j]=df_double.iloc[i,3+j].split(",")[1]

df_date=pd.DataFrame(columns=df_double.columns,index=range(2))

df_date.iloc[0,]=df_date.columns
df_date.iloc[1,]=df_date.columns

for i in range(len(df_date.columns)-3):
  df_date.iloc[0,3+i]=df_date.iloc[0,3+i].split(",")[0]
  df_date.iloc[1,3+i]=df_date.iloc[1,3+i].split(",")[1]

df_new=pd.concat([df_date,df_double])
df_new=df_new.reset_index(drop=True)

In [ ]:
df_new.to_csv('output.csv',encoding="cp932",index = False)
from google.colab import files
files.download('output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>